#### This text is much less organized than others. This is a 'play around' code used to help create 'Clean_Data_loan_amount.py'

Readers are suggested to see the python file located in the same folder, if they simply want to utilize the main parts of the code to extract the loan amount and currency information.

In [1]:
#! /usr/bin/env python3

from itertools import *
import os
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
#nltk.download()

import spacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# ## If loading in future, use this to load in the data without having to recompute everything
raw_df = pd.read_csv('../Clean_Data/clean_data.csv')
raw_df_tess = pd.read_csv('../Clean_Data/clean_data_tess.csv')

raw_df = raw_df.drop(columns=raw_df.columns[0])
raw_df_tess = raw_df_tess.drop(columns=raw_df_tess.columns[0])

In [18]:
# ## Path to text files
# path = "/home/kbari/git_repo/FinanceErdosProj/PyMuPdf_Text/"
# path1 = "/home/kbari/git_repo/FinanceErdosProj/Tesseract_Text/"

## Load from txt from files to a dataframe; Other information to include possibly?

## Path to text files (Kevin Bombardier)
path = "../PyMuPdf_Text/"
path1 = "../Tesseract_Text/"

In [3]:
## Load from txt from files to a dataframe; Other information to include possibly?

#pd.read_table(file,header=None,quotechar=None,quoting=3,error_bad_lines=False) for file in DIR]
def txt_to_df(path):
    ''' Put all txt files into single dataframe'''
    DIR = os.listdir(path)
    raw_df_lst = []
    for i in range(len(DIR)):
        with open(path+DIR[i], encoding="utf8") as f:
            lines = f.readlines()
            data = '\n'.join(map(str,lines))
#             print(loan_amount_preprocess(data))
#             print(country_preprocess(data))
            di = pd.DataFrame([data,len(data)],
                              index=['raw_text','num_char'],columns=[DIR[i]]).T
            raw_df_lst.append(di)
    #print(len(raw_df_lst))
    df_raw = pd.concat(raw_df_lst)
    return df_raw

In [31]:
# ## Get the raw dataframe using the function previously defined as txt_to_df
# ## Only used at the beginning. Use the read_csv version later
# raw_df = txt_to_df(path)
# raw_df_tess = txt_to_df(path1)

In [112]:
sample_df = raw_df.sample(n=5)

sample_df.raw_text

,raw_text,num_char,clean_text,filename,loan_amount_curr,loan_amount,currency
566,...,37885,"['conformed', 'copy', 'loan', 'number', 'ru', ...",1994_september_27_819981468294100608_conformed...,"[500000000, $]",500000000,$
2599,OFFICIAL \n\nDOCUMENTS \n\nLOAN NUMBER 8708-PA...,36773,"['official', 'document', 'loan', 'number', 'pa...",2017_july_7_415511500642367300_official-docume...,"[12000000, $]",12000000,$
844,CONFORMED COPY \n\nLOAN NUMBER 4124 CHA\n\nLo...,34201,"['conformed', 'copy', 'loan', 'number', 'cha',...",1997_august_29_225681468223152721_conformed-co...,"[400000000, $]",400000000,$
1861,CONFORMED COPY \n\n \n\nLOAN NUMBER 7667-UR \n...,30218,"['conformed', 'copy', 'loan', 'number', 'ur', ...",2009_february_10_860301468302438096_loan-agree...,Error,Error,NaN
1927,CONFORMED COPY \n\n \n\nLOAN NUMBER 7809-DO \n...,30573,"['conformed', 'copy', 'loan', 'number', 'do', ...",2009_november_18_836531468027660798_loan-agree...,"[15000000, $]",15000000,$


In [136]:
## Removes punctuation, stopwords, lemmatizes
def text_preprocess(text):
    '''
    Remove all punctuation,stopwords, lemmatize -> returns list of words
    '''
    lemma = WordNetLemmatizer()
    text = text.lower()
#     text = ''.join(char for char in text if char not in string.punctuation)
    text = ' '.join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text).split())    #keeps only alphabets and numbers. change
    text = ' '.join(i for i in text if not i.isdigit())
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))
    text = ' '.join(lemma.lemmatize(str(word)) for word in nlp(text))
    return text

In [137]:
sample_df['clean_data'] = sample_df.raw_text.apply(lambda x: text_preprocess(x))

sample_df

,raw_text,num_char,clean_text,filename,loan_amount_curr,loan_amount,currency,clean_data
566,...,37885,"['conformed', 'copy', 'loan', 'number', 'ru', ...",1994_september_27_819981468294100608_conformed...,"[500000000, $]",500000000,$,c n f r e c p l n n u b e r r u l n g r e e e ...
2599,OFFICIAL \n\nDOCUMENTS \n\nLOAN NUMBER 8708-PA...,36773,"['official', 'document', 'loan', 'number', 'pa...",2017_july_7_415511500642367300_official-docume...,"[12000000, $]",12000000,$,f f c l c u e n l n n u b e r p l n g r e e e ...
844,CONFORMED COPY \n\nLOAN NUMBER 4124 CHA\n\nLo...,34201,"['conformed', 'copy', 'loan', 'number', 'cha',...",1997_august_29_225681468223152721_conformed-co...,"[400000000, $]",400000000,$,c n f r e c p l n n u b e r c h l n g r e e e ...
1861,CONFORMED COPY \n\n \n\nLOAN NUMBER 7667-UR \n...,30218,"['conformed', 'copy', 'loan', 'number', 'ur', ...",2009_february_10_860301468302438096_loan-agree...,Error,Error,NaN,c n f r e c p l n n u b e r u r l n g r e e e ...
1927,CONFORMED COPY \n\n \n\nLOAN NUMBER 7809-DO \n...,30573,"['conformed', 'copy', 'loan', 'number', 'do', ...",2009_november_18_836531468027660798_loan-agree...,"[15000000, $]",15000000,$,c n f r e c p l n n u b e r l n g r e e e n p ...


In [ ]:
def clean_docs(doc):
    doc = doc.lower() #converts to lower  
    doc = ' '.join(lemmatizer(str(word), VERB)[0] for word in nlp(doc))    #lemmatizes the verbs
    doc = ' '.join(word.text for word in nlp(doc) if not word.is_stop)    #removes stopwords
    doc = ' '.join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", doc).split())    #keeps only alphabets and numbers. change it to [^A-Za-z \t}
    doc = doc.strip()
    return doc

In [35]:
## Will create a column called "clean_text" which uses Lemmatization to process raw text
raw_df['clean_text'] = raw_df.raw_text.apply(lambda x: text_preprocess(x))
raw_df_tess['clean_text'] = raw_df_tess.raw_text.apply(lambda x: text_preprocess(x))

In [40]:
raw_df['filename'] = raw_df.index
raw_df_tess['filename'] = raw_df_tess.index

In [38]:
df1 = raw_df.copy()
df2 = raw_df_tess.copy()

In [52]:
raw_df = raw_df.reset_index(drop=True)
raw_df_tess = raw_df_tess.reset_index(drop=True)

In [55]:
## Save the dataframes to the csv
raw_df.to_csv('../clean_data.csv')
raw_df_tess.to_csv('../clean_data_tess.csv')

In [5]:
def contains_number(reg_exp):
    y = [char.isdigit() for char in reg_exp]
    if y.count(True) > 5:
        return True
    return False

In [5]:
def txt_to_csv(path,out_path):
    '''Input path is path for all text files, output path is path to output
    file'''
    df_raw = txt_to_df(path)
    return df_raw.to_csv(out_path,index=False)

In [290]:
txt = raw_df.raw_text.iloc[545]

amount = re.findall("\((.+?\d+?)\)",txt)[0]
amount = amount.replace(',','')
amount = amount.replace('?','')
amount = amount.replace(' ','')
currency = re.findall("\D+",amount)[0]
amount = re.findall("\d+",amount)[0]
amount = int(amount)

In [333]:
txt = raw_df.raw_text.iloc[0]

for nums in re.findall("\((.+?\d+?)\)",txt):
    am = nums.replace(',','')
    am = am.replace('?','')
    am = am.replace(' ','')
    try:
        curr = re.findall("\D+",am)[0]
    except:
        # Most likely means there is no currency present
        continue
    am = re.findall("\d+",am)[0]
    am = int(am)
    print(am)

80000000
1


In [7]:
txt = 'aaaa kjlkjrelkwjriojsdfoijwiorjeqr'

re.findall('aaaa .{5}',txt)

['aaaa kjlkj']

In [4]:
def create_df_from_list(df, col_name):
    # Takes a column of lists from a dataframe
    # Returns a new dataframe with new columns the length of the lists
    
    temp_df = df[col_name].apply(pd.Series)
    temp_df = temp_df.rename(columns = lambda x: col_name + '_' + str(x))
    new_df = pd.concat([df, temp_df], axis=1)
    return new_df

In [5]:
# Will convert currencies to a standard form
def currency_corr(text):
    if ((re.search("US",text.upper())) or (re.search("\$",text)) or (re.search("dollar",text.lower()))):
        return '$' #US Dollar
    elif ((re.search("EUR",text.upper())) or (re.search("\€",text)) or (text.upper()=='E')):
        return 'EUR' #Euro
    elif (re.search("SDR",text.upper()) or (re.search("right",text.lower()))):
        return 'SDR' #Special Drawing Rights
    elif ((re.search("POUND",text.upper())) or (re.search("\£",text)) or (re.search("GBP",text.upper()))):
        return 'GBP' #British Pound
    elif (re.search("FRF",text.upper()) or (re.search("frank",text.lower()))):
        return 'FRF' #Franks
    elif ((re.search("JPY",text.upper())) or (re.search("\¥",text)) or (re.search("yen",text.lower()))):
        return 'JPY' #Japanese Yen
    elif ((re.search("DEM",text.upper())) or (re.search("DM",text.upper())) or (re.search("mark",text.lower()))):
        return 'DEM' #Deutsche Mark
    elif (re.search("INR",text.upper()) or (re.search("rupee",text.lower()))):
        return 'INR' #Indian Rupee
    elif (re.search("RM",text.upper()) or (re.search("ringgit",text.lower()))):
        return 'RM' #Malaysian Ringgit
    elif (re.search("SEK",text.upper()) or (re.search("krona",text.lower()))):
        return 'SEK' #Swedish Krona
    else:
        # No currency found, return what was input
        return text

In [ ]:
# def loan_amount_currency_preprocess(text):
#     ''' Grab Loan amount using reg exp'''
#     amount=0
#     curr='error'
#     currency='error'
    
#     # If there's the word "aggregate" in the document, assume it is the best indicator of the total loan amount.
#     # Look for a number and letters inside parentheses shortly after the mention of "aggregate".
#     # These most likely will be the aggregate loan with currency. If it's not in parentheses, then move on to another
#     # possible indicator method.
    
#     # This could possibly fail as other aspects of the document may mention "aggregate". ¯\_(ツ)_/¯
#     if re.search("AGGREGATE",text.upper()): # Determines if the exact word "aggregate" appears in the document
#         for txt in re.findall('AGGREGATE [\S\n ]{150}',text.upper()): # Loop through all instances and capture the next 200 chars
#             for nums in re.findall("\((.+?\d+?)\)",txt): # Find all numbers with other characters inside of parentheses
#                 am = nums.replace(',','')
#                 am = am.replace('?','')
#                 am = am.replace(' ','')
#                 try:
#                     curr = re.findall("\D+",am)[0]
#                 except:
#                     # Most likely means there is no currency present
#                     continue
#                 am = re.findall("\d+",am)[0]
#                 am = int(am)
                
#                 # Convert currencies
#                 curr = currency_corr(curr)
                
#                 # The document may fail to load the proper currency, try to grab the currency from the name beforehand
#                 if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
#                     and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
#                     try:
#                         curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
#                         curr = re.findall('\w+(?=\s+\()',curr)[0]
#                         curr = currency_corr(curr)
#                     except:
#                         pass
#                     try:
#                         curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
#                         curr = re.findall('\w+(?=\s+\()',curr)[0]
#                         curr = currency_corr(curr)
#                     except:
#                         pass

#                 # First check if it's USD, if it is, we're likely happy with it if it's large enough
#                 if ((curr=='$') and (am > 1000000)):
#                     return [am, curr]
#                 elif am > amount:
#                     amount = am
#                     currency = curr
    
#     # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
#     if (amount >= 1000000) and (len(currency)<5):
        
#         # Correct all currencies to proper forms
#         currency = currency_corr(currency)

#         # At this point, the currency probably isn't worth remembering
#         if ((currency != "$") and (currency != "EUR") and (currency != "SDR") and (currency != "GBP")
#             and (currency != "FRF") and (currency != "JPY") and (currency != "DEM") and (currency != "INR")
#             and (currency != "RM") and (currency != "SEK")):
#             return 'Error'
#         else:
#             return [amount, currency]
    
#     # Next tries to find the loan amount between Articles II and III
#     if (re.search('ARTICLE II ',text.upper()) or re.search('ARTICLE II\n',text.upper())):
#         spacer1 = 'ARTICLE II'
#     elif (re.search('ARTICLE 11 ',text.upper()) or re.search('ARTICLE 11\n',text.upper())):
#         spacer1 = 'ARTICLE 11'
#     elif (re.search('ARTICLE 2 ',text.upper()) or re.search('ARTICLE 2\n',text.upper())):
#         spacer1 = 'ARTICLE 2'
        
#     if (re.search('ARTICLE III ',text.upper()) or re.search('ARTICLE III\n',text.upper())):
#         spacer2 = 'ARTICLE II'
#     elif (re.search('ARTICLE 111 ',text.upper()) or re.search('ARTICLE 111\n',text.upper())):
#         spacer2 = 'ARTICLE 11'
#     elif (re.search('ARTICLE 3 ',text.upper()) or re.search('ARTICLE 3\n',text.upper())):
#         spacer2 = 'ARTICLE 3'
    
#     try:
#         reg_ex = spacer1 + ' [\S\n ]+ ' + spacer2
#     except:
#         reg_ex = 'ARTICLE II [\S\n ]+ ARTICLE' #Who knows what went wrong!
    
#     if re.search(reg_ex,text.upper()):
#         for txt in re.findall('[\S\n ]{100}\(.+?\d+?\)',re.findall(reg_ex,text.upper())[0]): # Loop through all instances
#             for nums in re.findall("\((.+?\d+?)\)",txt):
#                 am = nums.replace(',','')
#                 am = am.replace('?','')
#                 am = am.replace(' ','')
#                 try:
#                     curr = re.findall("\D+",am)[0]
#                 except:
#                     # Most likely means there is no currency present
#                     continue
#                 am = re.findall("\d+",am)[0]
#                 am = int(am)

#                 # Convert currencies
#                 curr = currency_corr(curr)

#                 # The document may fail to load the proper currency, try to grab the currency from the name beforehand
#                 if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
#                     and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
#                     try:
#                         curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
#                         curr = re.findall('\w+(?=\s+\()',curr)[0]
#                         curr = currency_corr(curr)
#                     except:
#                         pass
#                     try:
#                         curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
#                         curr = re.findall('\w+(?=\s+\()',curr)[0]
#                         curr = currency_corr(curr)
#                     except:
#                         pass
        
#                 if am > amount:
#                     amount = am
#                     currency = curr
    
#     # Likely preferable to a brute-force approach, so settle on this loan amount if reasonable
#     # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
#     if (amount >= 1000000) and (len(currency)<5):
#         # Correct all currencies to proper forms
#         currency = currency_corr(currency)
    
#         # At this point, the currency probably isn't worth remembering
#         if ((currency != "$") and (currency != "EUR") and (currency != "SDR") and (currency != "GBP")
#             and (currency != "FRF") and (currency != "JPY") and (currency != "DEM") and (currency != "INR")
#             and (currency != "RM") and (currency != "SEK")):
#             return 'Error'
#         else:
#             return [amount, currency]
    
#     # Brute force approach, find all numbers in the document in parentheses, find the largest one inside parentheses
#     for txt in re.findall('[\S\n ]{100}\(.+?\d+?\)',text): # Loop through all instances and capture the next 200
#         for nums in re.findall("\((.+?\d+?)\)",txt):
#             am = nums.replace(',','')
#             am = am.replace('?','')
#             am = am.replace(' ','')
#             try:
#                 curr = re.findall("\D+",am)[0]
#             except:
#                 # Most likely means there is no currency present
#                 continue
#             am = re.findall("\d+",am)[0]
#             am = int(am)
        
#             # Convert currencies
#             curr = currency_corr(curr)
        
#             # The document may fail to load the proper currency, try to grab the currency from the name beforehand
#             if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
#                 and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
#                 try:
#                     curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
#                     curr = re.findall('\w+(?=\s+\()',curr)[0]
#                     curr = currency_corr(curr)
#                 except:
#                     pass
#                 try:
#                     curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
#                     curr = re.findall('\w+(?=\s+\()',curr)[0]
#                     curr = currency_corr(curr)
#                 except:
#                     pass
        
#             if am > amount:
#                 amount = am
#                 currency = curr
    
#     # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
#     if (amount >= 1000000) and (len(currency)<5):
#         # Correct all currencies to proper forms
#         currency = currency_corr(currency)
#         # At this point, the currency probably isn't worth remembering
#         if ((currency != "$") and (currency != "EUR") and (currency != "SDR") and (currency != "GBP")
#             and (currency != "FRF") and (currency != "JPY") and (currency != "DEM") and (currency != "INR")
#             and (currency != "RM") and (currency != "SEK")):
#             return 'Error'
#         else:
#             return [amount, currency]
#     else:
#         return 'Error'

In [833]:
text = raw_df.raw_text.iloc[669]

amount=0
curr='error'
currency='error'

# First try to look between Articles II and III for the loan amount...
# Will try to capture the biggest number in Article II in hopes that is correct
if (re.search('ARTICLE II ',text.upper()) or re.search('ARTICLE II\n',text.upper())):
    spacer1 = 'ARTICLE II'
elif (re.search('ARTICLE 11 ',text.upper()) or re.search('ARTICLE 11\n',text.upper())):
    spacer1 = 'ARTICLE 11'
elif (re.search('ARTICLE 2 ',text.upper()) or re.search('ARTICLE 2\n',text.upper())):
    spacer1 = 'ARTICLE 2'

if (re.search('ARTICLE III ',text.upper()) or re.search('ARTICLE III\n',text.upper())):
    spacer2 = 'ARTICLE III'
elif (re.search('ARTICLE 111 ',text.upper()) or re.search('ARTICLE 111\n',text.upper())):
    spacer2 = 'ARTICLE 111'
elif (re.search('ARTICLE 3 ',text.upper()) or re.search('ARTICLE 3\n',text.upper())):
    spacer2 = 'ARTICLE 3'

try:
    reg_ex = spacer1 + '[\S\n ]*?' + '(THE LOAN[\S\n ]*?' + spacer2 + ')'
except:
    # Try to be less specific. Hope the first capture will work
    # First looks for two Articles, hoping to catch the 2nd article and then 'the loan' after it
    reg_ex = 'ARTICLE[\S\n ]*?ARTICLE[\S\n ]*?(THE LOAN[\S\n ]+?\(.+?\d+?\))'
    
print(reg_ex)
    
re.findall(reg_ex,text.upper())[0]

ARTICLE II[\S
 ]*?(THE LOAN[\S
 ]*?ARTICLE III)


'THE LOAN\n\nSECTION 2.01. \n\nTHE BANK AGREES TO LEND TO THE BORROWER, ON THE TERMS AND\n\nCONDITIONS SET FORTH OR REFERRED TO IN THE LOAN AGREEMENT, VARIOUS CURRENCIES THAT SHALL\n\nHAVE AN AGGREGATE VALUE EQUIVALENT TO THE AMOUNT OF THREE HUNDRED AND FIFTY MILLION DOLLARS\n\n($350,000,000), BEING THE SUM OF WITHDRAWALS OF THE PROCEEDS OF THE LOAN, WITH EACH\n\nWITHDRAWAL VALUED BY THE BANK AS OF THE DATE OF SUCH WITHDRAWAL.\n\nSECTION 2.02. \n\nTHE AMOUNT OF THE LOAN MAY BE WITHDRAWN FROM THE LOAN\n\nACCOUNT IN ACCORDANCE WITH THE PROVISIONS OF SCHEDULE 1 TO THIS AGREEMENT FOR AMOUNTS\n\nPAID (OR, IF THE BANK SHALL SO AGREE, TO BE PAID) BY THE BORROWER ON ACCOUNT OF WITHDRAWALS\n\nMADE BY AN INVESTMENT ENTERPRISE UNDER A SUB-LOAN TO MEET THE REASONABLE COST OF GOODS\n\nAND SERVICES REQUIRED FOR THE INVESTMENT PROJECT IN RESPECT OF WHICH THE WITHDRAWAL FROM\n\nTHE LOAN ACCOUNT IS REQUESTED.\n\nSECTION 2.03. \n\nTHE CLOSING DATE SHALL BE OCTOBER 31, 2000, OR SUCH LATER DATE AS\n\nTHE 

In [834]:
# # Used for testing purposes

# text = raw_df.raw_text.iloc[669]

# amount=0
# curr='error'
# currency='error'

# # First try to look between Articles II and III for the loan amount...
# # Will try to capture the biggest number in Article II in hopes that is correct
# if (re.search('ARTICLE II ',text.upper()) or re.search('ARTICLE II\n',text.upper())):
#     spacer1 = 'ARTICLE II'
# elif (re.search('ARTICLE 11 ',text.upper()) or re.search('ARTICLE 11\n',text.upper())):
#     spacer1 = 'ARTICLE 11'
# elif (re.search('ARTICLE 2 ',text.upper()) or re.search('ARTICLE 2\n',text.upper())):
#     spacer1 = 'ARTICLE 2'

# if (re.search('ARTICLE III ',text.upper()) or re.search('ARTICLE III\n',text.upper())):
#     spacer2 = 'ARTICLE III'
# elif (re.search('ARTICLE 111 ',text.upper()) or re.search('ARTICLE 111\n',text.upper())):
#     spacer2 = 'ARTICLE 111'
# elif (re.search('ARTICLE 3 ',text.upper()) or re.search('ARTICLE 3\n',text.upper())):
#     spacer2 = 'ARTICLE 3'

# try:
#     reg_ex = spacer1 + '[\S\n ]*?' + '(THE LOAN[\S\n ]*?' + spacer2 + ')'
# except:
#     # Try to be less specific. Hope the first capture will work
#     # First looks for two Articles, hoping to catch the 2nd article and then 'the loan' after it
#     reg_ex = 'ARTICLE[\S\n ]*?ARTICLE[\S\n ]*?(THE LOAN[\S\n ]+?\(.+?\d+?\))'
    
# print(reg_ex)
    
# if re.search(reg_ex,text.upper()):
#     for txt in re.findall('[\S\n ]{50}\(.+?\d+?\)',re.findall(reg_ex,text.upper())[0]): # Loop through all instances
#         for nums in re.findall("\((.+?\d+?)\)",txt):
#             am = nums.replace(',','')
#             am = am.replace('?','')
#             am = am.replace(' ','')
#             try:
#                 curr = re.findall("\D+",am)[0]
#             except:
#                 # Most likely means there is no currency present
#                 continue
#             am = re.findall("\d+",am)[0]
#             am = int(am)

#             # Convert currencies
#             curr = currency_corr(curr)

#             # The document may fail to load the proper currency, try to grab the currency from the name beforehand
#             if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
#                 and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
#                 try:
#                     curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
#                     curr = re.findall('\w+(?=\s+\()',curr)[0]
#                     curr = currency_corr(curr)
#                 except:
#                     pass
#                 try:
#                     curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
#                     curr = re.findall('\w+(?=\s+\()',curr)[0]
#                     curr = currency_corr(curr)
#                 except:
#                     pass

#             if am > amount:
#                 amount = am
#                 currency = curr

# # Likely preferable to a brute-force approach, so settle on this loan amount if reasonable
# # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
# if (amount >= 1000000) and (len(currency)<5):
#     # Correct all currencies to proper forms
#     currency = currency_corr(currency)

#     # At this point, the currency probably isn't worth remembering
#     if ((currency == "$") or (currency == "EUR") or (currency == "SDR") or (currency == "GBP")
#         or (currency == "FRF") or (currency == "JPY") or (currency == "DEM") or (currency == "INR")
#         or (currency == "RM") or (currency == "SEK")):
#         print([amount, currency])

ARTICLE II[\S
 ]*?(THE LOAN[\S
 ]*?ARTICLE III)
[350000000, '$']


In [6]:
def loan_amount_currency_preprocess(text):
    ''' Grab Loan amount using reg exp'''
    amount=0
    curr='error'
    currency='error'
    
    # First try to look between Articles II and III for the loan amount...
    # Will try to capture the biggest number in Article II in hopes that is correct
    if (re.search('ARTICLE II ',text.upper()) or re.search('ARTICLE II\n',text.upper())):
        spacer1 = 'ARTICLE II'
    elif (re.search('ARTICLE 11 ',text.upper()) or re.search('ARTICLE 11\n',text.upper())):
        spacer1 = 'ARTICLE 11'
    elif (re.search('ARTICLE 2 ',text.upper()) or re.search('ARTICLE 2\n',text.upper())):
        spacer1 = 'ARTICLE 2'
        
    if (re.search('ARTICLE III ',text.upper()) or re.search('ARTICLE III\n',text.upper())):
        spacer2 = 'ARTICLE III'
    elif (re.search('ARTICLE 111 ',text.upper()) or re.search('ARTICLE 111\n',text.upper())):
        spacer2 = 'ARTICLE 111'
    elif (re.search('ARTICLE 3 ',text.upper()) or re.search('ARTICLE 3\n',text.upper())):
        spacer2 = 'ARTICLE 3'
    
    try:
        reg_ex = spacer1 + '[\S\n ]*?' + '(THE LOAN[\S\n ]*?' + spacer2 + ')'
    except:
        # Try to be less specific. Hope the first capture will work
        # First looks for two Articles, hoping to catch the 2nd article and then 'the loan' after it
        reg_ex = 'ARTICLE[\S\n ]*?ARTICLE[\S\n ]*?(THE LOAN[\S\n ]+?\(.+?\d+?\))'
    
    if re.search(reg_ex,text.upper()):
        for txt in re.findall('[\S\n ]{50}\(.+?\d+?\)',re.findall(reg_ex,text.upper())[0]): # Loop through all instances
            for nums in re.findall("\((.+?\d+?)\)",txt):
                am = nums.replace(',','')
                am = am.replace('?','')
                am = am.replace(' ','')
                try:
                    curr = re.findall("\D+",am)[0]
                except:
                    # Most likely means there is no currency present
                    continue
                am = re.findall("\d+",am)[0]
                am = int(am)

                # Convert currencies
                curr = currency_corr(curr)

                # The document may fail to load the proper currency, try to grab the currency from the name beforehand
                if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
                    and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
                    try:
                        curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
                        curr = re.findall('\w+(?=\s+\()',curr)[0]
                        curr = currency_corr(curr)
                    except:
                        pass
                    try:
                        curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
                        curr = re.findall('\w+(?=\s+\()',curr)[0]
                        curr = currency_corr(curr)
                    except:
                        pass
        
                if am > amount:
                    amount = am
                    currency = curr
    
    # Likely preferable to a brute-force approach, so settle on this loan amount if reasonable
    # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
    if (amount >= 1000000) and (len(currency)<5):
        # Correct all currencies to proper forms
        currency = currency_corr(currency)
    
        # At this point, the currency probably isn't worth remembering
        if ((currency == "$") or (currency == "EUR") or (currency == "SDR") or (currency == "GBP")
            or (currency == "FRF") or (currency == "JPY") or (currency == "DEM") or (currency == "INR")
            or (currency == "RM") or (currency == "SEK")):
            return [amount, currency]
    
    # This could possibly fail as other aspects of the document may mention "aggregate". ¯\_(ツ)_/¯
    # Prefer finding the word 'aggregate' for the loan amount
    if re.search("AGGREGATE",text.upper()): # Determines if the exact word "aggregate" appears in the document
        for txt in re.findall('AGGREGATE [\S\n ]{150}',text.upper()): # Loop through all instances and capture the next 200 chars
            for nums in re.findall("\((.+?\d+?)\)",txt): # Find all numbers with other characters inside of parentheses
                am = nums.replace(',','')
                am = am.replace('?','')
                am = am.replace(' ','')
                try:
                    curr = re.findall("\D+",am)[0]
                except:
                    # Most likely means there is no currency present
                    continue
                am = re.findall("\d+",am)[0]
                am = int(am)
                
                # Convert currencies
                curr = currency_corr(curr)
                
                # The document may fail to load the proper currency, try to grab the currency from the name beforehand
                if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
                    and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
                    try:
                        curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
                        curr = re.findall('\w+(?=\s+\()',curr)[0]
                        curr = currency_corr(curr)
                    except:
                        pass
                    try:
                        curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
                        curr = re.findall('\w+(?=\s+\()',curr)[0]
                        curr = currency_corr(curr)
                    except:
                        pass

                # First check if it's USD, if it is, we're likely happy with it if it's large enough
                if ((curr=='$') and (am > 1000000)):
                    return [am, curr]
                elif am > amount:
                    amount = am
                    currency = curr
    
    # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
    if (amount >= 1000000) and (len(currency)<5):
        
        # Correct all currencies to proper forms
        currency = currency_corr(currency)

        # At this point, the currency probably isn't worth remembering
        if ((currency == "$") or (currency == "EUR") or (currency == "SDR") or (currency == "GBP")
            or (currency == "FRF") or (currency == "JPY") or (currency == "DEM") or (currency == "INR")
            or (currency == "RM") or (currency == "SEK")):
            return [amount, currency]
    
    # Brute force approach, find all numbers in the document in parentheses, find the largest one inside parentheses
    for txt in re.findall('[\S\n ]{100}\(.+?\d+?\)',text): # Loop through all instances and capture the next 200
        for nums in re.findall("\((.+?\d+?)\)",txt):
            am = nums.replace(',','')
            am = am.replace('?','')
            am = am.replace(' ','')
            try:
                curr = re.findall("\D+",am)[0]
            except:
                # Most likely means there is no currency present
                continue
            am = re.findall("\d+",am)[0]
            am = int(am)
        
            # Convert currencies
            curr = currency_corr(curr)
        
            # The document may fail to load the proper currency, try to grab the currency from the name beforehand
            if ((curr != "$") and (curr != "EUR") and (curr != "SDR") and (curr != "GBP") and (curr != "FRF") 
                and (curr != "JPY") and (curr != "DEM") and (curr != "INR") and (curr != "RM") and (curr != "SEK")):
                try:
                    curr = re.findall('MILLION([\S\n ]+\()',txt.upper())[0]
                    curr = re.findall('\w+(?=\s+\()',curr)[0]
                    curr = currency_corr(curr)
                except:
                    pass
                try:
                    curr = re.findall('BILLION([\S\n ]+\()',txt.upper())[0]
                    curr = re.findall('\w+(?=\s+\()',curr)[0]
                    curr = currency_corr(curr)
                except:
                    pass
        
            if am > amount:
                amount = am
                currency = curr
    
    # Sanity check. Currency should be reasonable in length and amount should be at least 1 million
    if (amount >= 1000000) and (len(currency)<5):
        # Correct all currencies to proper forms
        currency = currency_corr(currency)
        # At this point, the currency probably isn't worth remembering
        if ((currency != "$") and (currency != "EUR") and (currency != "SDR") and (currency != "GBP")
            and (currency != "FRF") and (currency != "JPY") and (currency != "DEM") and (currency != "INR")
            and (currency != "RM") and (currency != "SEK")):
            return 'Error'
        else:
            return [amount, currency]
    else:
        return 'Error'

In [7]:
# ## If loading in future, use this to load in the data without having to recompute everything
raw_df = pd.read_csv('../Clean_Data/clean_data.csv')
raw_df_tess = pd.read_csv('../Clean_Data/clean_data_tess.csv')

raw_df = raw_df.drop(columns=raw_df.columns[0])
raw_df_tess = raw_df_tess.drop(columns=raw_df_tess.columns[0])

In [8]:
raw_df['loan_amount_curr'] = raw_df.raw_text.apply(lambda x: loan_amount_currency_preprocess(x))

In [9]:
raw_df_tess['loan_amount_curr'] = raw_df_tess.raw_text.apply(lambda x: loan_amount_currency_preprocess(x))

In [10]:
raw_df = create_df_from_list(raw_df, 'loan_amount_curr')
raw_df = raw_df.rename(columns={'loan_amount_curr_0': 'loan_amount', 'loan_amount_curr_1': 'currency'})

In [11]:
raw_df_tess = create_df_from_list(raw_df_tess, 'loan_amount_curr')
raw_df_tess = raw_df_tess.rename(columns={'loan_amount_curr_0': 'loan_amount', 'loan_amount_curr_1': 'currency'})

In [12]:
raw_df.currency.value_counts()

$      2312
EUR     282
DEM      31
SDR      22
FRF      21
JPY      10
GBP       2
INR       1
SEK       1
Name: currency, dtype: int64

In [13]:
raw_df_tess.currency.value_counts()

$      331
EUR     42
SDR      3
JPY      1
GBP      1
Name: currency, dtype: int64

In [11]:
raw_df.columns

Index(['raw_text', 'num_char', 'clean_text', 'filename', 'loan_amount_curr',
       'loan_amount', 'currency'],
      dtype='object')

In [16]:
raw_df[['filename', 'loan_amount', 'currency']].to_csv('loan_amount.csv', index=False)

In [17]:
raw_df_tess[['filename', 'loan_amount', 'currency']].to_csv('loan_amount_tess.csv', index=False)

In [18]:
df = pd.concat([raw_df, raw_df_tess])
df[['filename', 'loan_amount', 'currency']].to_csv('loan_data.csv', index=False)

In [14]:
len(raw_df[raw_df.loan_amount_curr == 'Error'])

117

In [15]:
len(raw_df_tess[raw_df_tess.loan_amount == 'Error'])

22

In [68]:
# # Possibility 2

# def loan_amount_currency_preprocess(text):
#     ''' Grab Loan amount using reg exp'''
#     try:
#         ## Searches to see if 'Article II' is present in the document, if not, will try another form
#         # NOTE THE SPACE AND \n IS IMPORTANT AFTER II
        
#         if (re.search('ARTICLE II ',text.upper()) or re.search('ARTICLE II\n',text.upper())):
#             spacer = 'ARTICLE II'
#         elif (re.search('ARTICLE 11 ',text.upper()) or re.search('ARTICLE 11\n',text.upper())):
#             spacer = 'ARTICLE 11'
#         elif (re.search('ARTICLE 2 ',text.upper()) or re.search('ARTICLE 2\n',text.upper())):
#             spacer = 'ARTICLE 2'
        
#         t1 = text.split(spacer)[1]
#         amount = re.findall("\((.+?\d+?)\)",t1)[0]
#         amount = amount.replace(',','')
#         amount = amount.replace('?','')
#         amount = amount.replace(' ','')
#         currency = re.findall("\D+",amount)[0]
#         amount = re.findall("\d+",amount)[0]
#         amount = int(amount)
        
#         ## Some sanity checks. If these fail, then the loan amounts or currencies grabbed are not reasonable.
#         ## Attempt other methods to grab loan amount and currency upon these failures
        
#         # The loan amount is too small if this check passes or the currency is weird and unreasonable
#         if (amount < 100000 or len(currency)>=8):
#             curr='error'
#             # Brute force approach, find all numbers in the document, find the largest one inside parentheses
#             for nums in re.findall("\((.+?\d+?)\)",text):
#                 am = nums.replace(',','')
#                 am = am.replace('?','')
#                 am = am.replace(' ','')
#                 try:
#                     curr = re.findall("\D+",am)[0]
#                 except:
#                     # Most likely means there is no currency present
#                     continue
#                 am = re.findall("\d+",am)[0]
#                 am = int(am)
#                 if am > amount:
#                     amount = am
#                     currency = curr

#         return [amount, currency]
#     except:
#         # If there's any error, return an error (for checking effectiveness)
#         return 'error'

In [535]:
len(new_df_tess.filename)

378

In [843]:
# new_df = raw_df[raw_df.loan_amount_curr != 'Error']
new_df = raw_df

In [844]:
# new_df_tess = raw_df_tess[raw_df_tess.loan_amount_curr != 'Error']
new_df_tess = raw_df_tess

In [708]:
new_df.iloc[accuracy_check_index]

,raw_text,num_char,clean_text,filename,loan_amount_curr,loan_amount,currency
4,CONFO...,33593,"['conformed', 'copy', 'loan', 'number', 'ind',...",1990_april_30_725911468042268845_conformed-cop...,"[350000000, $]",350000000,$
6,CONFORMED COPY\n\nLOAN NUMBER 3203 KO\n\n ...,26018,"['conformed', 'copy', 'loan', 'number', 'ko', ...",1990_august_10_460651468271855106_conformed-co...,"[45000000, $]",45000000,$
7,...,18685,"['conformed', 'copy', 'loan', 'number', 'ko', ...",1990_august_10_885551468047414878_conformed-co...,"[31600000, $]",31600000,$
8,...,30971,"['conformed', 'copy', 'loan', 'number', 'ko', ...",1990_august_10_939121468273631507_conformed-co...,"[34000000, $]",34000000,$
26,...,44157,"['loan', 'number', 'yu', 'loan', 'agreement', ...",1990_february_1_328621468247829056_conformed-c...,"[28000000, $]",28000000,$
...,...,...,...,...,...,...,...
2757,OFFICIAL\n\nDOCUMENTS\n\n国■ ■■ ■ ■ ■ ■■ ■国 国■■...,18747,"['official', 'document', '国■', '■■', '■', '■',...",2018_november_5_261671546532139420_official-do...,"[34440000, $]",34440000,$
2769,OFFICIAL\n\nDOCUMENTS\n\nLOAN NUMBER 8867-AR\n...,32720,"['official', 'document', 'loan', 'number', 'ar...",2019_january_31_727951551717070453_official-do...,"[150000000, $]",150000000,$
2770,OFFICIAL\n\nDOCUMENTS\n\nLOAN NUMBER 8974-TR\n...,29559,"['official', 'document', 'loan', 'number', 'tr...",2019_july_10_527321563303180984_official-docum...,"[500000000, EUR]",500000000,EUR
2795,OFFICIAL\n\nDOCUMENTS\n\nLOAN NUMBER 9014-BY\n...,28038,"['official', 'document', 'loan', 'number', 'by...",2019_november_28_539191576006466769_official-d...,"[90000000, EUR]",90000000,EUR


In [699]:
new_df[new_df.index.isin([accuracy_check_index.tolist()])==True]

,raw_text,num_char,clean_text,filename,loan_amount_curr,loan_amount,currency


In [698]:
len(new_df.index)

2681

In [697]:
accuracy_check_index

0         4
1         6
2         7
3         8
4        26
       ... 
295    2757
296    2769
297    2770
298    2795
299    2798
Length: 300, dtype: int64

In [691]:
type(new_df.index)

pandas.core.indexes.numeric.Int64Index

In [845]:
accuracy_check = new_df.iloc[accuracy_check_index][['filename', 'loan_amount', 'currency']].sort_index()
accuracy_check_tess = new_df_tess.iloc[accuracy_check_tess_index][['filename', 'loan_amount', 'currency']].sort_index()

In [846]:
accuracy_check.to_csv('../accuracy_check.csv')
accuracy_check_tess.to_csv('../accuracy_check_tess.csv')

In [561]:
accuracy_check_tess_index = pd.Series(accuracy_check_tess.index)
accuracy_check_index = pd.Series(accuracy_check.index)

In [559]:
pd.Series(accuracy_check.index).to_csv('../accuracy_check_index.csv', index=False)
pd.Series(accuracy_check_tess.index).to_csv('../accuracy_check_tess_index.csv', index=False)